

<p style="font-family: Arial; font-size:2.4em;color:green;"> Importing libraries </p>

In [ ]:
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
# pip install featuretools
import featuretools as ft

<p style="font-family: Arial; font-size:2.4em;color:green;"> Read Data and Parse Date Column </p>

In [85]:
# Parse_dates argument converts 2016 06 10 20:30:00 to 2016-06-10 by default if no function is
# passed to the date_parser argument of pd.read_csv() method.

clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])

In [86]:
clients.head()


,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [87]:
loans.sample(5)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
384,46958,cash,6596,1,10537,2005-08-25,2008-03-29,2.48
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
335,39384,credit,2166,1,10274,2012-02-27,2013-08-30,1.56
260,29841,cash,4301,1,10742,2005-01-15,2007-07-18,3.68
52,41480,home,9766,0,10567,2008-06-03,2010-01-08,9.08


In [88]:
loans.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   client_id    443 non-null    int64         
 1   loan_type    443 non-null    object        
 2   loan_amount  443 non-null    int64         
 3   repaid       443 non-null    int64         
 4   loan_id      443 non-null    int64         
 5   loan_start   443 non-null    datetime64[ns]
 6   loan_end     443 non-null    datetime64[ns]
 7   rate         443 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(4), object(1)
memory usage: 27.8+ KB


In [39]:
payments.sample(5)

,loan_id,payment_amount,payment_date,missed
2058,10798,236,2009-05-25,1
1830,10851,1803,2009-11-28,1
1069,11418,432,2006-11-22,1
1736,11113,2489,2002-01-07,1
2133,11610,1565,2011-02-16,1


<p style="font-family: Arial; font-size:2.0em;color:green;"> Some Transformations On Clients DF </p>

In [89]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column to get income in the same range as other numerical range.
clients['log_income'] = np.log(clients['income'])

# Observe the new clients dataframe after transformations.
clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


<p style="font-family: Arial; font-size:1.4em;color:green;"> Aggregating Loan Amount Stats for Each Client . The resultant Stats dataframe will be merged with the clients dataframe.</p>

In [90]:
# Aggregating Information from loans dataframe for each client.
# Groupby client id and calculate mean, max, min previous loan size in loan dataframe.
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [91]:
# To check that every customer in clients table has a record in loans table.
# If client_id count in clients df == no. of records in stats table then Okay,
# Otherwise some rows will have null value in the clients and stats merged df  
n = clients.nunique(axis=0)
print(n)
m = stats.nunique(axis=0)
print(m)

client_id       25
joined          24
income          25
credit_score    25
join_month       9
log_income      25
dtype: int64
mean_loan_amount    25
max_loan_amount     25
min_loan_amount     25
dtype: int64


<p style="font-family: Arial; font-size:2.0em;color:green;"> Merge stats Df with clients df. </p>

In [92]:
# Merge aggregated information with the clients dataframe
# stats 
clients1 = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(5)


In [94]:
# We could go further and include information about payments in the clients dataframe. To do so, 
# we would have to group payments by the loan_id, merge it with the loans, group the resulting dataframe 
# by the client_id, and then merge it into the clients dataframe. This would allow us to include 
# information about previous payments for each client.

# Print the merged stats dataframe with client infor dataframe
clients1

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.60,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.30,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.85,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.85,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.95,13913,1212


<p style="font-family: Arial; font-size:1.8em;color:green;"> Creating Entity Set using Featuretools module for aggregated data between related tables  </p>

In [95]:
# Entity sets will be used to define relationships between client , loans and payments dataframes and then 
# finding meaningfull aggregation data for each clients.
es = ft.EntitySet(id = 'clients')

In [ ]:
# 
from woodwork.logical_types import Categorical

<p style="font-family: Arial; font-size:1.8em;color:green;"> Adding Clients Dataframe to entity set and defining parameters</p>

In [97]:
# Parameters Definitions
# 1. dataframe_name --> A Unique key to access the dataframe from the Entity set
# 2. dataframe --> Dataframe variable.
# 3. index     --> A column which uniquely identifies every row in the dataframe.
# 4. time_index --> Any Datetime related column names are given here
# 5. logical_types -->  parameter indicates that “repaid” column in loans df should be interpreted as a Categorical column, even though it is just an integer in the underlying data.

es = es.add_dataframe(
    dataframe_name="clients",
    dataframe=clients,
    index="client_id",
    time_index="joined",
)

<p style="font-family: Arial; font-size:1.8em;color:green;"> Adding Loans Dataframe to entity set and defining parameters</p>

In [98]:
es = es.add_dataframe(
    dataframe_name="loans",
    dataframe=loans,
    index="loan_id",
    time_index="loan_start",
    logical_types={
        "repaid": Categorical,
    },
)


<p style="font-family: Arial; font-size:1.8em;color:green;"> Adding Payments Dataframe to entity set and defining parameters</p>

In [99]:
es = es.add_dataframe(
    dataframe_name="payments",
    dataframe=payments,
    make_index = True,
    index="payment_id",
    time_index="payment_date",
    logical_types={
        "missed": Categorical,
    },
)

In [100]:
# A method to access a particular dataframe from the entity set just like dictionaries.
es['loans']

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
11140,39505,home,2274,1,11140,2000-01-26,2002-01-29,1.00
11251,26326,home,2847,1,11251,2000-03-06,2001-09-26,1.32
10816,49545,home,8354,1,10816,2000-03-08,2001-08-02,0.45
11965,29841,credit,6012,0,11965,2000-03-25,2002-07-10,4.63
10166,41472,home,13657,1,10166,2000-04-11,2001-09-08,5.68
...,...,...,...,...,...,...,...,...
11595,35089,other,773,1,11595,2014-09-26,2016-04-23,7.63
10985,26695,other,13918,1,10985,2014-10-03,2016-10-25,0.90
10684,48177,credit,659,1,10684,2014-10-05,2017-01-16,1.52
10131,49068,other,10082,1,10131,2014-10-10,2016-05-25,0.63


<p style="font-family: Arial; font-size:2.4em;color:green;">Defining Relationships between different dataframes in an Entity Set </p>

<p style="font-family: Arial; font-size:1.8em;color:green;"> Mapping client_id of Clients df with client_id of loans df.</p>

In [101]:
# Syntax is ( parent_df , parent_df_index , child_df , child_df_index ) 
es = es.add_relationship("clients", "client_id", "loans", "client_id")
# Check relationship being added under the Entity set.
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id

<p style="font-family: Arial; font-size:1.8em;color:green;"> Mapping loan_id of loans df with loan_id of payments df</p>

In [102]:
# Syntax is ( parent_df , parent_df_index , child_df , child_df_index ) 
es = es.add_relationship("loans", "loan_id", "payments", "loan_id")
# Check relationship being added under the Entity set.
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

<p style="font-family: Arial; font-size:1.8em;color:green;">Feature Primitives</p>

<p style="font-family: Arial; font-size:1.4em;color:green;">A feature primitive a at a very high-level is an operation applied to data to create a feature. These represent very simple calculations that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:                1. Aggregation  and 
2. Transformation</p>

<p style="font-family: Arial; font-size:1.8em;color:green;">Some Aggregation Feature Primitives</p>

In [128]:
# Viewing the list of Aggregration feature primitives along with their usecases 
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,dask_compatible,koalas_compatible,description,valid_inputs,return_type
0,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` values.",<ColumnSchema (Semantic Tags = ['numeric'])>,None
1,num_true,aggregation,True,False,Counts the number of `True` values.,"<ColumnSchema (Logical Type = BooleanNullable)>, <ColumnSchema (Logical Type = Boolean)>",None
2,min,aggregation,True,True,"Calculates the smallest value, ignoring `NaN` values.",<ColumnSchema (Semantic Tags = ['numeric'])>,None
3,median,aggregation,False,False,Determines the middlemost number in a list of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,None
4,mean,aggregation,True,True,Computes the average for a list of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,None
5,num_unique,aggregation,True,True,"Determines the number of distinct values, ignoring `NaN` values.",<ColumnSchema (Semantic Tags = ['category'])>,None
6,sum,aggregation,True,True,"Calculates the total addition, ignoring `NaN`.",<ColumnSchema (Semantic Tags = ['numeric'])>,None
7,all,aggregation,True,False,Calculates if all values are 'True' in a list.,"<ColumnSchema (Logical Type = BooleanNullable)>, <ColumnSchema (Logical Type = Boolean)>",None
8,skew,aggregation,False,False,Computes the extent to which a distribution differs from a normal distribution.,<ColumnSchema (Semantic Tags = ['numeric'])>,None
9,first,aggregation,False,False,Determines the first value in a list.,<ColumnSchema>,None


<p style="font-family: Arial; font-size:1.8em;color:green;">Some Transformation Feature Primitives</p>

In [129]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,dask_compatible,koalas_compatible,description,valid_inputs,return_type
22,hour,transform,True,True,Determines the hour value of a datetime.,<ColumnSchema (Logical Type = Datetime)>,None
23,not_equal,transform,True,False,Determines if values in one list are not equal to another list.,<ColumnSchema>,None
24,equal,transform,True,True,Determines if values in one list are equal to another list.,<ColumnSchema>,None
25,less_than,transform,True,True,Determines if values in one list are less than another list.,"<ColumnSchema (Semantic Tags = ['numeric'])>, <ColumnSchema (Logical Type = Ordinal)>, <ColumnSc...",None
26,negate,transform,True,True,Negates a numeric value.,<ColumnSchema (Semantic Tags = ['numeric'])>,None
27,not,transform,True,True,Negates a boolean value.,"<ColumnSchema (Logical Type = BooleanNullable)>, <ColumnSchema (Logical Type = Boolean)>",None
28,url_to_tld,transform,False,False,Determines the top level domain of a url.,<ColumnSchema (Logical Type = URL)>,None
29,day,transform,True,True,Determines the day of the month from a datetime.,<ColumnSchema (Logical Type = Datetime)>,None
30,cum_sum,transform,False,False,Calculates the cumulative sum.,<ColumnSchema (Semantic Tags = ['numeric'])>,None
31,divide_numeric_scalar,transform,True,True,Divide each element in the list by a scalar.,<ColumnSchema (Semantic Tags = ['numeric'])>,None


<p style="font-family: Arial; font-size:1.4em;color:green;">Trying Primitives using the ft.dfs function (which stands for deep feature synthesis) to extract primitives related data on various columns for all clients in the clients dataframe.</p>

In [103]:
features, feature_names = ft.dfs(entityset = es, target_dataframe_name = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['month','weekday'])

<p style="font-family: Arial; font-size:1.8em;color:green;">Using Trans Primitives on 'joined' column of clients df</p>

In [104]:
# Using trans primitive 'weekday' on joined column of clients dataframe.
trans_weekday_on_joined_column = pd.DataFrame(features['WEEKDAY(joined)'])

In [107]:
trans_weekday_on_joined_column

,WEEKDAY(joined)
client_id,
42320,3
39384,6
26945,6
41472,1
46180,1
46109,1
32885,0
29841,5
38537,0


In [ ]:
# Using trans primitive 'weekday' on joined column of clients dataframe.
trans_month_on_joined_column = pd.DataFrame(features['MONTH(joined)'])

In [ ]:
trans_month_on_joined_column

<p style="font-family: Arial; font-size:1.8em;color:green;">Using Agg. Primitives on loan_column of loans df . Results are aggregated on basis of index column of target_dataframe_name parameter of ft.dfs(). </p>

In [119]:
aggregate_res = pd.DataFrame(features['MAX(loans.loan_amount)'])

In [120]:
aggregate_res

,MAX(loans.loan_amount)
client_id,
42320,13887.0
39384,14654.0
26945,14593.0
41472,13657.0
46180,14081.0
46109,14049.0
32885,14162.0
29841,14837.0
38537,14804.0


<p style="font-family: Arial; font-size:1.8em;color:green;">Automated Deep Feature Synthesis</p>


<p style="font-family: Arial; font-size:1.2em;color:green;">
In addition to manually specifying aggregation and transformation feature primitives, we can let feature tools automatically generate many new features. We do this by making the same ft.dfs function call, but without passing in any primitives. We just set the max_depth parameter and feature tools will automatically try many all combinations of feature primitives to the ordered depth.</p>

In [121]:
# For this call, we only need to specify the entityset, 
# the target_entity (which will again be clients), and the max_depth.
# The parameter max_depth controls the complexity of the features being generated.

features_automated, feature_names_automated = ft.dfs(entityset=es, target_dataframe_name='clients', 
                                 max_depth = 2)

In [122]:
# Getting a subset of the deep feature dataframe returned in the previous step.
features_automated.iloc[:, 4:].head()

,COUNT(loans),MAX(loans.loan_amount),MAX(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),MODE(loans.loan_type),MODE(loans.repaid),NUM_UNIQUE(loans.loan_type),...,MODE(payments.loans.loan_type),MODE(payments.loans.repaid),NUM_UNIQUE(payments.loans.loan_type),NUM_UNIQUE(payments.loans.repaid),SKEW(payments.loans.loan_amount),SKEW(payments.loans.rate),STD(payments.loans.loan_amount),STD(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,15,13887.0,6.74,7062.066667,2.457333,1070.0,0.38,home,1,4,...,home,1,4,2,0.097213,0.872916,3929.351652,1.891948,843367.0,302.84
39384,19,14654.0,9.23,7865.473684,3.538421,1770.0,0.43,credit,1,4,...,cash,1,4,2,-0.277360,0.971757,3765.368569,2.685701,1161741.0,499.12
26945,15,14593.0,5.65,7125.933333,2.855333,653.0,0.13,credit,0,4,...,credit,0,4,2,0.289824,0.020986,4372.558827,1.599556,771053.0,330.13
41472,16,13657.0,9.82,7510.812500,3.981250,986.0,0.01,cash,0,4,...,other,1,4,2,-0.064213,0.311596,4250.127784,3.129716,784731.0,435.36
46180,20,14081.0,9.26,7700.850000,3.502500,1607.0,0.57,other,0,4,...,other,0,4,2,0.127445,0.649145,3815.702946,2.714351,1142666.0,578.43


<p style="font-family: Arial; font-size:1.8em;color:green;">Conclusion</p>


<p style="font-family: Arial; font-size:1.2em;color:green;">In this notebook, we saw how to apply automated feature engineering to an example dataset. This is a powerful method which allows us to overcome the human limits of time and imagination to create many new features from multiple tables of data.</p>

<p style="font-family: Arial; font-size:1.8em;color:green;">Next Steps </p>

<p style="font-family: Arial; font-size:1.2em;color:green;"> Feature engineering allows us to combine information across many tables into a single dataframe that we can then use for machine learning model training. Finally, the next step after creating all of these features is figuring out which ones are important using methods discussed in earlier sessions. </p>